In [ ]:
# 토크나이저 첫걸음 노트북

- 순서대로 따라가면 **라이브러리 없이도** 기본 토큰화를 체험할 수 있습니다.
- 마지막에는 `tokenizers`/`sentencepiece` 같은 라이브러리를 어떻게 설치해 쓰는지도 안내합니다.
- 셀마다 주석으로 "무엇을 하면 되는지" 적어두었으니 그대로 실행만 해보세요.


In [2]:
# Step 0. 실험 문장 정의 (자유롭게 수정 가능)
sentences = [
    "해리포터는 마법사야!",
    "AI 모델은 다음 단어를 예측해요.",
    "새로운_토큰?? 안녕🙂"
]

sentences


['해리포터는 마법사야!', 'AI 모델은 다음 단어를 예측해요.', '새로운_토큰?? 안녕🙂']

In [3]:
import re
from typing import List


def tokenize_whitespace(text: str) -> List[str]:
    return text.split()


def tokenize_characters(text: str) -> List[str]:
    return list(text)


def tokenize_punct_aware(text: str) -> List[str]:
    pattern = r"[A-Za-z0-9_]+|[\uAC00-\uD7A3]+|[^\sA-Za-z0-9_\uAC00-\uD7A3]"
    return re.findall(pattern, text)


def show_all_basic(text: str) -> None:
    print(f"문장: {text}")
    ws = tokenize_whitespace(text)
    print("- 공백 기준:", ws)
    chars = tokenize_characters(text)
    print("- 문자 기준(앞부분):", chars[:40], ("..." if len(chars) > 40 else ""))
    punct = tokenize_punct_aware(text)
    print("- 구두점 인지:", punct)
    print()

for s in sentences:
    show_all_basic(s)


문장: 해리포터는 마법사야!
- 공백 기준: ['해리포터는', '마법사야!']
- 문자 기준(앞부분): ['해', '리', '포', '터', '는', ' ', '마', '법', '사', '야', '!'] 
- 구두점 인지: ['해리포터는', '마법사야', '!']

문장: AI 모델은 다음 단어를 예측해요.
- 공백 기준: ['AI', '모델은', '다음', '단어를', '예측해요.']
- 문자 기준(앞부분): ['A', 'I', ' ', '모', '델', '은', ' ', '다', '음', ' ', '단', '어', '를', ' ', '예', '측', '해', '요', '.'] 
- 구두점 인지: ['AI', '모델은', '다음', '단어를', '예측해요', '.']

문장: 새로운_토큰?? 안녕🙂
- 공백 기준: ['새로운_토큰??', '안녕🙂']
- 문자 기준(앞부분): ['새', '로', '운', '_', '토', '큰', '?', '?', ' ', '안', '녕', '🙂'] 
- 구두점 인지: ['새로운', '_', '토큰', '?', '?', '안녕', '🙂']



## 라이브러리 쓰면 뭐가 좋을까?
- `tokenizers` (Hugging Face)나 `sentencepiece` 라이브러리를 쓰면 **BPE, WordPiece, Unigram** 같은 실제 알고리즘을 쉽게 체험할 수 있어요.
- 설치 명령 예시 (가상환경에서 실행 권장):
  ```bash
  pip install tokenizers sentencepiece
  ```
- 아래 셀은 `tokenizers`가 설치되어 있다는 가정 하에 돌아갑니다. 없으면 먼저 위 명령을 실행하세요.


In [4]:
try:
    from tokenizers import Tokenizer, models, trainers, pre_tokenizers
except ImportError as e:
    raise SystemExit("tokenizers 라이브러리가 설치되어 있지 않습니다. 위의 pip 명령을 먼저 실행하세요.") from e

# Step 1. 간단한 말뭉치 준비 (실제론 더 크고 다양해야 함)
corpus = [
    "해리포터는 마법사야!",
    "마법사는 지팡이로 주문을 외운다.",
    "AI 모델은 토큰을 숫자로 바꾼다.",
    "SentencePiece와 BPE는 서브워드를 만든다.",
]

# Step 2. BPE 모델 생성
bpe_model = models.BPE(unk_token="[UNK]")
trainer = trainers.BpeTrainer(vocab_size=100, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]"])
tokenizer = Tokenizer(bpe_model)
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.train_from_iterator(corpus, trainer=trainer)

print("=== BPE vocab 일부 ===")
print(list(tokenizer.get_vocab().keys())[:20])

print("\n=== 샘플 문장 토크나이즈 ===")
for sentence in sentences:
    output = tokenizer.encode(sentence)
    print(sentence)
    print("토큰:", output.tokens)
    print("토큰 ID:", output.ids)
    print()





=== BPE vocab 일부 ===
['P', '[UNK]', '로', '사', 't', '바', '이로', 'c', '든', 'B', '마법', '만든다', '리', '지', '델은', '이', '외운다', '숫', '다', '지팡이로']

=== 샘플 문장 토크나이즈 ===
해리포터는 마법사야!
토큰: ['해리포터는', '마법사야', '!']
토큰 ID: [94, 86, 4]

AI 모델은 다음 단어를 예측해요.
토큰: ['AI', '모델은', '다', '[UNK]', '[UNK]', '[UNK]', '를', '[UNK]', '[UNK]', '해', '[UNK]', '.']
토큰 ID: [57, 70, 19, 1, 1, 1, 24, 1, 1, 52, 1, 5]

새로운_토큰?? 안녕🙂
토큰: ['[UNK]', '로', '운', '[UNK]', '토', '큰', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]']
토큰 ID: [1, 23, 39, 1, 49, 47, 1, 1, 1, 1, 1]



### 정리 메모
- **라이브러리 없이**도 공백/문자/구두점 단위로 쪼개보며 감을 잡을 수 있다.
- **라이브러리 사용** 시에는 BPE 같은 알고리즘이 "자주 나오는 조각"을 자동으로 vocab에 넣어 준다.
- vocab 크기나 말뭉치를 바꾸면 토큰 수와 조각이 어떻게 달라지는지 직접 실험해보자.
